In [1]:
import guidance  # requires Guidance with main branch and PR https://github.com/guidance-ai/guidance/pull/348 to be merged

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

apitype = os.getenv("OPENAI_API_TYPE")
apibase = os.getenv("OPENAI_API_BASE")
apikey = os.getenv("OPENAI_API_KEY")

model = "gpt-4"

guidance.llm = guidance.llms.OpenAI(
    model = model, 
    deployment_id = model,
    api_type = apitype, 
    api_base = apibase,
    api_key = apikey,
    api_version = "2023-05-15")


In [4]:
from suggesters.simple_model_suggester import SimpleModelSuggester
modeler = SimpleModelSuggester()

# Test pairwise relationships

In [7]:
result = modeler.suggest_pairwise_relationship("ice cream sales", "shark attacks")

#result = modeler.suggest_pairwise_relationship("ice cream sales", "temperature")

result

Stop program system You are a helpful assistant for causal reasoning. user Which cause-and-effect-relationship is more likely?
 A. ice cream sales causes shark attacks 
 B. shark attacks causes ice cream sales 
 C. neither ice cream sales nor shark attacks cause each other
 First let's very succinctly think about each option.
 Then I'll ask you to provide your final answer A, B, or C. assistant A. Ice cream sales causing shark attacks: This seems unlikely. There's no logical connection between the act of selling ice cream and the behavior of sharks. 

B. Shark attacks causing ice cream sales: This also seems unlikely. While it's possible that a community might rally around a local business after a traumatic event like a shark attack, it's not a direct cause-and-effect relationship.

C. Neither ice cream sales nor shark attacks cause each other: This seems the most likely. Ice cream sales and shark attacks are both events that can occur independently of each other. They might both increase during the summer months, for example, but one does not cause the other.

Final answer: C. Neither ice cream sales nor shark attacks cause each other. user Now what is your final answer: A, B, or C? assistant C

[None,
 None,
 "A. Ice cream sales causing shark attacks: This seems unlikely. There's no logical connection between the act of selling ice cream and the behavior of sharks. \n\nB. Shark attacks causing ice cream sales: This also seems unlikely. While it's possible that a community might rally around a local business after a traumatic event like a shark attack, it's not a direct cause-and-effect relationship.\n\nC. Neither ice cream sales nor shark attacks cause each other: This seems the most likely. Ice cream sales and shark attacks are both events that can occur independently of each other. They might both increase during the summer months, for example, but one does not cause the other.\n\nFinal answer: C. Neither ice cream sales nor shark attacks cause each other."]

In [8]:
if result[0] is not None:
    print(f"{result[0]} causes {result[1]}")
else:
    print(f"neither causes the other")

neither causes the other


# Let's build a graph among multiple variables

In [7]:
variables = ["ice cream sales", "temperature", "cavities"]
results = modeler.suggest_relationships(variables)

results

Stop program system You are a helpful assistant for causal reasoning. user Which cause-and-effect-relationship is more likely?
 A. temperature causes cavities 
 B. cavities causes temperature 
 C. neither temperature nor cavities cause each other
 First let's very succinctly think about each option.
 Then I'll ask you to provide your final answer A, B, or C. assistant A. Temperature causes cavities: This is unlikely. While extreme temperatures can cause discomfort or sensitivity in teeth, especially if cavities are already present, there's no scientific evidence to suggest that temperature itself causes cavities. Cavities are primarily caused by factors such as poor oral hygiene, frequent snacking, sugary drinks, and not getting enough fluoride.

B. Cavities cause temperature: This is also unlikely. Cavities themselves do not cause changes in body temperature. However, if a cavity becomes infected, it could potentially lead to a fever as the body fights off the infection. But it's important to note that the fever is a response to the infection, not the cavity itself.

C. Neither temperature nor cavities cause each other: This seems to be the most likely scenario. While temperature can affect the comfort of teeth and cavities, and an infected cavity can lead to a fever, there's no direct cause-and-effect relationship between temperature and cavities.

Final Answer: C. Neither temperature nor cavities cause each other. user Now what is your final answer: A, B, or C? assistant C

	No relationship found between temperature and cavities


{('temperature',
  'ice cream sales'): "A. Ice cream sales causing temperature: This implies that the number of ice cream sales would directly affect the temperature. This is unlikely as the sale of a product, such as ice cream, does not have the ability to influence weather patterns or temperature changes.\n\nB. Temperature causing ice cream sales: This implies that as the temperature increases, more people would want to buy ice cream to cool down. This is a more plausible scenario as people's buying habits, especially for food and beverages, can be influenced by the weather.\n\nC. Neither ice cream sales nor temperature cause each other: This implies that there is no relationship between the two. While it's possible that in some cases there might not be a direct correlation, generally, it's reasonable to assume that hot weather can influence ice cream sales.\n\nFinal Answer: B. Temperature causes ice cream sales.",
 ('ice cream sales',
  'cavities'): "A. Ice cream sales causes caviti

In [6]:
results

{('temperature',
  'ice cream sales'): "A. Ice cream sales causing temperature: This implies that the number of ice cream sales would directly affect the temperature. This is unlikely as the sale of a product, such as ice cream, does not have the ability to influence weather patterns or temperature changes.\n\nB. Temperature causing ice cream sales: This implies that as the temperature increases, more people would want to buy ice cream to cool down. This is a more plausible scenario as people's buying habits, especially for food and beverages, can be influenced by the weather.\n\nC. Neither ice cream sales nor temperature cause each other: This implies that there is no relationship between the two. While it's possible that in some cases there might not be a direct correlation, generally, it's reasonable to assume that hot weather can influence ice cream sales.\n\nFinal Answer: B. Temperature causes ice cream sales.",
 ('ice cream sales',
  'cavities'): "A. Ice cream sales causes caviti

# Latent confounders

In [9]:
variables = ["ice cream sales", "temperature", "cavities"]
latents = modeler.suggest_confounders(variables, treatment="ice cream sales", outcome = "shark attacks")

print(latents)

Stop program system You are a helpful assistant for causal reasoning. user What latent confounding factors might influence the relationship between ice cream sales and shark attacks ?

 We have already considered the following factors ['ice cream sales', 'temperature', 'cavities'] . Please do not repeat them.

 List the confounding factors between ice cream sales and shark attacks enclosing the name of each factor in <conf> </conf> tags. assistant <conf>Beach Attendance</conf>, <conf>Season of the Year</conf>, <conf>Water Temperature</conf>, <conf>Public Holidays</conf>, <conf>Availability of Ice Cream Vendors</conf>, <conf>Shark Population</conf>, <conf>Swimming Conditions</conf>, <conf>Tourist Season</conf>.

['Beach Attendance', 'Season of the Year', 'Water Temperature', 'Public Holidays', 'Availability of Ice Cream Vendors', 'Shark Population', 'Swimming Conditions', 'Tourist Season']


In [10]:
latents = modeler.suggest_confounders([], treatment="vitamin c", outcome = "cardiovascular health")

print(latents)

Stop program system You are a helpful assistant for causal reasoning. user What latent confounding factors might influence the relationship between vitamin c and cardiovascular health ?

 We have already considered the following factors [] . Please do not repeat them.

 List the confounding factors between vitamin c and cardiovascular health enclosing the name of each factor in <conf> </conf> tags. assistant <conf>Dietary Habits</conf>: The type of food a person consumes can influence both their vitamin C intake and their cardiovascular health. 

<conf>Physical Activity Level</conf>: Regular exercise can improve cardiovascular health and may also influence dietary choices, including foods rich in vitamin C.

<conf>Smoking Status</conf>: Smoking can deplete the body's vitamin C levels and is also a risk factor for cardiovascular disease.

<conf>Alcohol Consumption</conf>: Alcohol can affect the absorption of vitamin C and is also linked to cardiovascular health.

<conf>Genetic Factors</conf>: Certain genetic factors can influence how the body metabolizes vitamin C and how susceptible a person is to cardiovascular disease.

<conf>Age</conf>: Aging can affect both cardiovascular health and the body's need for and absorption of vitamin C.

<conf>Chronic Diseases</conf>: Conditions like diabetes or kidney disease can affect cardiovascular health and may also influence how the body processes vitamin C.

<conf>Medication Use</conf>: Some medications can affect cardiovascular health and may also interact with vitamin C.

<conf>Socioeconomic Status</conf>: This can influence access to nutritious food (including those high in vitamin C) and healthcare, which in turn can affect cardiovascular health.

<conf>Stress Levels</conf>: Chronic stress can impact both cardiovascular health and the body's use of nutrients like vitamin C.

['Dietary Habits', 'Physical Activity Level', 'Smoking Status', 'Alcohol Consumption', 'Genetic Factors', 'Age', 'Chronic Diseases', 'Medication Use', 'Socioeconomic Status', 'Stress Levels']


# Identification support

## Instrumental variables

In [12]:
from suggesters import SimpleIdentificationSuggester
identifier = SimpleIdentificationSuggester()

In [13]:
variables = ["cigarette taxes", "rain", "car sales", "property taxes", "heart attacks"]
ivs = identifier.suggest_iv(variables, 
                            treatment="smoking", outcome = "birth weight")

print(ivs)

Stop program system You are a helpful assistant for causal reasoning. user Which factors in ['cigarette taxes', 'rain', 'car sales', 'property taxes', 'heart attacks'] might be valid instrumental variables for identifying the effect of smoking on birth weight ?

 List the factors that are possible instrumental variables in <iv> </iv> tags. assistant <iv>'cigarette taxes'</iv>

["'cigarette taxes'"]


## Backdoor variables

In [8]:
variables = ["Age", "Sex", "HbA1c", "HDL", "LDL", "eGFR", "Prior MI", 
             "Prior Stroke or TIA", "Prior Heart Failure", "Cardiovascular medication", 
             "T2DM medication", "Insulin", "Morbid obesity", "First occurrence of Nonfatal myocardial infarction, nonfatal stroke, death from all cause", 
             "semaglutide treatment", "Semaglutide medication", "income", "musical taste"]

backdoors = identifier.suggest_backdoor(variables, 
                            treatment="semaglutide treatment", outcome = "cardiovascular health")

print(backdoors)

Stop program system You are a helpful assistant for causal reasoning. user Which set or subset of factors in ['Age', 'Sex', 'HbA1c', 'HDL', 'LDL', 'eGFR', 'Prior MI', 'Prior Stroke or TIA', 'Prior Heart Failure', 'Cardiovascular medication', 'T2DM medication', 'Insulin', 'Morbid obesity', 'First occurrence of Nonfatal myocardial infarction, nonfatal stroke, death from all cause', 'semaglutide treatment', 'Semaglutide medication', 'income', 'musical taste'] might satisfy the backdoor criteria for identifying the effect of semaglutide treatment on cardiovascular health ?

 List the factors satisfying the backdoor criteria enclosing the name of each factor in <backdoor> </backdoor> tags. assistant The backdoor criterion is a method to identify a valid set of variables for adjustment to estimate the causal effect of a treatment on an outcome. The backdoor criterion requires that no backdoor path from treatment to outcome is left open after conditioning on the selected variables. A backdoor path is a path that starts with an arrow into the treatment.

In this case, the treatment is 'semaglutide treatment' and the outcome is 'cardiovascular health'. The backdoor paths could be closed by conditioning on a set of variables that block all paths from the treatment to the outcome that go through an arrow into the treatment. 

Based on the given variables, the potential factors that might satisfy the backdoor criteria could be:

<backdoor>'Age'</backdoor>, <backdoor>'Sex'</backdoor>, <backdoor>'HbA1c'</backdoor>, <backdoor>'HDL'</backdoor>, <backdoor>'LDL'</backdoor>, <backdoor>'eGFR'</backdoor>, <backdoor>'Prior MI'</backdoor>, <backdoor>'Prior Stroke or TIA'</backdoor>, <backdoor>'Prior Heart Failure'</backdoor>, <backdoor>'Cardiovascular medication'</backdoor>, <backdoor>'T2DM medication'</backdoor>, <backdoor>'Insulin'</backdoor>, <backdoor>'Morbid obesity'</backdoor>.

Please note that the actual set of variables satisfying the backdoor criteria would depend on the specific causal structure of the variables, which is not provided here. Therefore, this is a potential list and might need to be adjusted based on the actual causal relationships among the variables.

# frontdoor

In [10]:
frontdoors = identifier.suggest_frontdoor(variables, 
                            treatment="semaglutide treatment", outcome = "cardiovascular health")

print(frontdoors)

Stop program system You are a helpful assistant for causal reasoning. user Which set or subset of factors in ['Age', 'Sex', 'HbA1c', 'HDL', 'LDL', 'eGFR', 'Prior MI', 'Prior Stroke or TIA', 'Prior Heart Failure', 'Cardiovascular medication', 'T2DM medication', 'Insulin', 'Morbid obesity', 'First occurrence of Nonfatal myocardial infarction, nonfatal stroke, death from all cause', 'semaglutide treatment', 'Semaglutide medication', 'income', 'musical taste'] might satisfy the frontdoor criteria for identifying the effect of semaglutide treatment on cardiovascular health ?

 List the factors satisfying the frontdoor criteria enclosing the name of each factor in <backdoor> </backdoor> tags. assistant The frontdoor criterion is a method used in causal inference to estimate the causal effect of a variable on an outcome. It requires a variable (or set of variables) that is affected by the treatment and only affects the outcome through the treatment. 

In this case, the treatment is 'semaglutide treatment' and the outcome is 'cardiovascular health'. The variables that might satisfy the frontdoor criteria could be:

<backdoor>'HbA1c'</backdoor>, <backdoor>'HDL'</backdoor>, <backdoor>'LDL'</backdoor>, <backdoor>'eGFR'</backdoor>, <backdoor>'Prior MI'</backdoor>, <backdoor>'Prior Stroke or TIA'</backdoor>, <backdoor>'Prior Heart Failure'</backdoor>, <backdoor>'Cardiovascular medication'</backdoor>, <backdoor>'T2DM medication'</backdoor>, <backdoor>'Insulin'</backdoor>, <backdoor>'Morbid obesity'</backdoor>, <backdoor>'First occurrence of Nonfatal myocardial infarction, nonfatal stroke, death from all cause'</backdoor>, <backdoor>'Semaglutide medication'</backdoor>.

These factors are related to both the treatment (semaglutide) and the outcome (cardiovascular health), and they are likely to be affected by the treatment and only affect the outcome through the treatment. However, the exact set of variables that satisfy the frontdoor criteria would depend on the specific causal structure of the problem, which is not provided here.

[]
